<a href="https://colab.research.google.com/github/ChrisRed255/Vehicle-Detection/blob/master/Vehicle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vehicle detection algorithm

**Import packages**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import keras
import pickle
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

**Prepare data for model**

In [0]:
DATADIR = os.path.join(os.getcwd(),os.path.join("Workspace","train"))
CATEGORY = ['Car','Landscape']
IMG_SIZE = 400
training_data =[]

def create_training_data():
  print("Processing ...\n\n")
  for category in CATEGORY:
    path = os.path.join(DATADIR,category)
    class_num = CATEGORY.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        training_data.append([new_array,class_num])
      except Exception as e:
        print("because of the category :"+category+", file : "+img+"\n"+str(e))
  print("Processed successfully !\n\n")

create_training_data()

**Shuffle data**

In [0]:
for i in range(0,6):
  random.shuffle(training_data)

**Create matrix X that holds features and matrix Y that holds labels**

In [0]:
X = []
y = []

for features,label in training_data:
  X.append(features)
  y.append(label)

**Reshape the features matrix**

In [0]:
X.shape

(4718, 400, 400, 3)

In [0]:
X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)

**Save X and Y into pickle file to use it after**

In [0]:
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

**Load pickle data X and Y**

In [0]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

**Normalize data**

In [0]:
#X = keras.utils.normalize(X, axis=-1, order=2)
X = X/255.0

**Create the model 1**

In [0]:
model = Sequential()
try:
  model.add(Conv2D(32,(3,3),input_shape = (500,500,3)))
  model.add(Activation("relu"))
  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  model.add(Conv2D(32,(5,5)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(64))
  model.add(Dropout(0.3))
  
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

except Exception as e:
  print("Error :",str(e))

In [0]:
#model.summary()
with open('modelInitial.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

**Create model 2**

In [0]:
model = Sequential()
try:
  model.add(Conv2D(32,(3,3),input_shape = X.shape[1:]))
  model.add(Activation("relu"))
  model.add(Conv2D(32,(1,1)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  model.add(Conv2D(32,(1,1)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3)))
  model.add(Activation("relu"))
  model.add(Conv2D(64,(3,3)))
  model.add(Activation("relu"))
  
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))
  
  model.add(Flatten())
  model.add(Dense(2))
  model.add(Activation('softmax'))

except Exception as e:
  print("Error :",str(e))

**Compile the model**

In [0]:
model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

**Train the model**

In [0]:
model.fit(X,y,epochs=20,validation_split=0.1)

**Save the model**

In [0]:
model.save('model.h5')

**Load the model**

In [0]:
model = tf.keras.models.load_model("model.h5")